In [1]:

import os
import time
import tracemalloc
import numpy as np
from guppy import hpy
hp = hpy()
file = open('GRCh38_latest_genomic.fna')

In [2]:
data = file.readlines()
file.close()

In [3]:
import math
pitch='>NC_000006.12 Homo sapiens chromosome 6, GRCh38.p13 Primary Assembly\n'
id=data.index(pitch)
for i in range(id,id+5000):
    if 'ttggtaccat' in data[i]:
        print(i,data[i])
    elif 'CTTTGCCTG' in data[i]:
        print(i,data[i])
begin_line = 13285205
end_line  =  13286454

data = data[begin_line:end_line+1]

13285205 ttggtaccattccttctgaaactattccaaacaacagaaaaagagagaatcctccctaactcattttatgaggccagaat

13286454 TTCCCTGAGCTGCTGTCTGGTCCAGAGCCCATGCACACAGGTGGAGACTGACTCTCCCTTAGGACCACTGCCTTTGCCTG

13288045 GGCTGGGTAGCCCCCAGTTTAATGCAGCCCCAGAAATGCCCAGCATCTTTGCCTTGTCTTTGCCTGTCATGGTTTTCTGT

13288441 GAAATGTGCTATTCGGAGCTTTGCCTGTGACCCCCTGCTTCGGAAGCATTCCTGCCCATGTGCCCCACTGATCTCTGCTT



# Order 0

In [4]:

start_time = time.time()
total=100000
a_num=0
t_num=0
c_num=0
g_num=0
for line_data in data:
    for letter in line_data.lower():
        if letter == 'a':
            a_num+=1
        elif letter == 't':
            t_num+=1
        elif letter == 'c':
            c_num+=1
        elif letter == 'g':
            g_num+=1
# print(a_num,t_num,c_num,g_num,a_num+t_num+c_num+g_num)

a_probability=a_num/total
t_probability=t_num/total
c_probability=c_num/total
g_probability=g_num/total
print(a_probability,t_probability,c_probability,g_probability)
order_0 = math.log2(a_probability)*a_num + math.log2(t_probability)*t_num +math.log2(c_probability)*c_num + math.log2(g_probability)*g_num

end_time = time.time()
print(order_0)
print(f'time{end_time - start_time} sec')
h=hp.heap()
print(h)

0.31515 0.26886 0.20803 0.20796
-197688.94757656846
time0.012996435165405273 sec
Partition of a set of 287181 objects. Total size = 35098283 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0  81142  28 10295875  29  10295875  29 str
     1  80719  28  6050792  17  16346667  47 tuple
     2  35335  12  2613388   7  18960055  54 bytes
     3  17790   6  2573360   7  21533415  61 types.CodeType
     4  15904   6  2162944   6  23696359  68 function
     5   2216   1  2121464   6  25817823  74 type
     6   5442   2  2017000   6  27834823  79 dict (no owner)
     7    859   0  1318688   4  29153511  83 dict of module
     8   2216   1  1075736   3  30229247  86 dict of type
     9  14445   5   417352   1  30646599  87 int
<839 more rows. Type e.g. '_.more' to view.>


# Order 1

In [5]:
start_time = time.time()
#initial
alphabet=['a','t','c','g']
combination={}#4*4種組合
for letter_1 in alphabet:
    for letter_2 in alphabet:
        combination[letter_1+letter_2]=0

#split data
split_data=''
for line_data in data:
    line_data = line_data.lower()
    split_data += line_data[0:len(line_data)-1]#不包含endline
#statistic
for index in range(len(split_data)-1):#不包含最後一個letter
        letter_1 = split_data[index] #假設at 中的a
        letter_2 = split_data[index+1] #假設at 中的t
        combination[letter_1+letter_2]+=1
print(combination)
#calculate
total={'a':0,'t':0,'c':0,'g':0}#開頭a or t or c or g
for first,second in combination:
    total[first]+=combination[first+second]
order_1=0
for fir,sec in combination:# if at: fir=a sec=t
    order_1 += math.log2(combination[fir+sec]/total[fir])*combination[fir+sec]

order_1+=t_probability

end_time = time.time()

print(order_1)
print(f'time{end_time - start_time} sec')

h=hp.heap()
print(h)

{'aa': 11197, 'at': 7576, 'ac': 5537, 'ag': 7205, 'ta': 6241, 'tt': 7909, 'tc': 5546, 'tg': 7190, 'ca': 7796, 'ct': 6656, 'cc': 5308, 'cg': 1043, 'ga': 6281, 'gt': 4744, 'gc': 4412, 'gg': 5358}
-193209.0890669481
time0.038895368576049805 sec
Partition of a set of 287432 objects. Total size = 35232038 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0  81178  28 10407290  30  10407290  30 str
     1  80743  28  6052208  17  16459498  47 tuple
     2  35348  12  2616188   7  19075686  54 bytes
     3  17790   6  2573360   7  21649046  61 types.CodeType
     4  15919   6  2164984   6  23814030  68 function
     5   2216   1  2121464   6  25935494  74 type
     6   5451   2  2019696   6  27955190  79 dict (no owner)
     7    859   0  1321112   4  29276302  83 dict of module
     8   2216   1  1075736   3  30352038  86 dict of type
     9  14467   5   417972   1  30770010  87 int
<847 more rows. Type e.g. '_.more' to view.>


# Order 2

In [6]:
start_time = time.time()
#initial
alphabet=['a','t','c','g']
combination={}#4*4*4種組合
for letter_1 in alphabet:
    for letter_2 in alphabet:
        for letter_3 in alphabet:
            combination[letter_1+letter_2+letter_3]=0

#split data
split_data=''
for line_data in data:
    line_data = line_data.lower()
    split_data += line_data[0:len(line_data)-1]#不包含endline
#statistic
for index in range(len(split_data)-2):#不包含最後一個即倒數第二個letter
        letter_1 = split_data[index] #假設atc 中的a
        letter_2 = split_data[index+1] #假設atc 中的t
        letter_3 = split_data[index+2] #假設atc 中的c
        combination[letter_1+letter_2+letter_3]+=1
print(combination)
#calculate
total={}
for i in alphabet:
    for j in alphabet:
        total[i+j]=0

for i in combination:
    total[i[0:2]]+=combination[i]

order_2=0
for i in combination:
    order_2 += math.log2(combination[i]/total[i[0:2]])*combination[i]
order_2 += t_probability

end_time = time.time()

print(order_2)
print(f'time{end_time - start_time} sec')
h=hp.heap()
print(h)

{'aaa': 4651, 'aat': 2624, 'aac': 1720, 'aag': 2202, 'ata': 2138, 'att': 2215, 'atc': 1289, 'atg': 1934, 'aca': 2342, 'act': 1647, 'acc': 1265, 'acg': 283, 'aga': 2398, 'agt': 1517, 'agc': 1493, 'agg': 1797, 'taa': 2177, 'tat': 1791, 'tac': 1147, 'tag': 1126, 'tta': 1755, 'ttt': 2807, 'ttc': 1756, 'ttg': 1591, 'tca': 2019, 'tct': 1857, 'tcc': 1447, 'tcg': 223, 'tga': 1974, 'tgt': 1863, 'tgc': 1454, 'tgg': 1898, 'caa': 2182, 'cat': 1807, 'cac': 1656, 'cag': 2151, 'cta': 1279, 'ctt': 1694, 'ctc': 1566, 'ctg': 2117, 'cca': 1931, 'cct': 1750, 'ccc': 1348, 'ccg': 279, 'cga': 256, 'cgt': 257, 'cgc': 235, 'cgg': 295, 'gaa': 2187, 'gat': 1354, 'gac': 1014, 'gag': 1726, 'gta': 1069, 'gtt': 1192, 'gtc': 935, 'gtg': 1548, 'gca': 1504, 'gct': 1402, 'gcc': 1248, 'gcg': 258, 'gga': 1653, 'ggt': 1107, 'ggc': 1230, 'ggg': 1368}
-192094.49773269478
time0.05585074424743652 sec
Partition of a set of 287566 objects. Total size = 35247399 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / di

In [7]:
transition_state_0=[0.4,0.6] #[from 0 to 0 , from 0 to 1]
transition_state_1=[0.7,0.3] #[from 1 to 0 , from 1 to 1]
emission_satte_0=[0.3,0.3,0.2,0.2]#[a,t,c,g]
emission_satte_1=[0.4,0.4,0.1,0.1]#[a,t,c,g]

In [8]:
# file = open('GRCh38_latest_genomic.fna')

In [9]:
# data = file.read()
# file.close()

In [10]:
# pitch='>NC_000006.12 Homo sapiens chromosome 6, GRCh38.p13 Primary Assembly'
# pitch2='>NC_000007.14 Homo sapiens chromosome 7, GRCh38.p13 Primary Assembly'
# file_name_index_begin= data.index(pitch)
# file_name_index_end= file_name_index_begin + 69# 68 word + 1 endline

# num_redundant_N=0
# for i,data_temp in enumerate(data[file_name_index_end+1:]):#+1 for first N
#     if data_temp!='N' and data_temp!='\n':
#         num_redundant_N=i+1 #i begin from 0
#         break
        
# data_begin=file_name_index_end+num_redundant_N #endline後第一個gene
# # print(data_begin,data[data_begin:data_begin+68])
# print(data[data_begin+10000:data_begin+200005].index('ttggtaccat'))

In [11]:
states = ('state_0', 'state_1')

split_data=''
for line_data in data:
    line_data = line_data.lower()
    split_data += line_data[0:len(line_data)-1]#不包含endline

observations = split_data
 
start_probability = {'state_0': 0.6, 'state_1': 0.4}
 
transition_probability = {
   'state_0' : {'state_0': 0.7, 'state_1': 0.3},
   'state_1' : {'state_0': 0.9, 'state_1': 0.1},
   }
 
emission_probability = {
   'state_0' : {'a': 0.4, 't': 0.1, 'c': 0.3,'g':0.2},
   'state_1' : {'a': 0.2, 't': 0.3, 'c': 0.2,'g':0.4},
   }

In [12]:
import math

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    path = {}

    # Initialize base cases (t == 0)
    for y in states:
        V[0][y] = math.log2(start_p[y] * emit_p[y][obs[0]])
        path[y] = [y]
    # Run Viterbi for t > 0
    for gene in range(1,len(obs)):
        V.append({})
        newpath = {}

        for y in states:
            (prob, state) = max([(V[gene-1][y0] + math.log2(trans_p[y0][y]) + math.log2(emit_p[y][obs[gene]]), y0) for y0 in states])
            V[gene][y] = prob
            newpath[y] = path[state] + [y]

        # Don't need to remember the old paths
        path = newpath

    #print_dptable(V)
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])
    return (prob, path[state])
#     return (prob)

In [14]:
start_time = time.time()

def example():
    return viterbi(observations,
                   states,
                   start_probability,
                   transition_probability,
                   emission_probability)



print('probability: ', example()[0])

end_time = time.time()
print(f'time{end_time - start_time} sec')
h=hp.heap()
print(h)

probability:  -250252.33071276834
time26.604285717010498 sec
Partition of a set of 287278 objects. Total size = 35228929 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0  81337  28 10434920  30  10434920  30 str
     1  80722  28  6051392  17  16486312  47 tuple
     2  35353  12  2614569   7  19100881  54 bytes
     3  17794   6  2573984   7  21674865  62 types.CodeType
     4  15904   6  2162944   6  23837809  68 function
     5   2216   1  2121848   6  25959657  74 type
     6   5446   2  2021224   6  27980881  79 dict (no owner)
     7    859   0  1321112   4  29301993  83 dict of module
     8   2216   1  1075736   3  30377729  86 dict of type
     9  14529   5   419708   1  30797437  87 int
<840 more rows. Type e.g. '_.more' to view.>
